## This script is for all the 6 gene signatures
1. Merge the clustering results
2. Extract the samples always high

In [1]:
import os
import re
import pandas as pd

In [15]:
series = 'tcga' # mondor

if series == 'tcga':
    path = '/media/visiopharm5/WDGold/deeplearning/Hepatocarcinomes/TCGA/validation_336_filter_before'
    first_signature = 'Interferon_Gamma_Biology'
elif series == 'mondor':
    path = '/media/visiopharm5/WDGold/deeplearning/Hepatocarcinomes/TCGA/heatmap_nanostring_139'
    first_signature = '6G_Interferon_Gamma'
    drop_list = ['full', 'selected']
else:
    raise NotImplementedError

In [9]:
signatures = []
for root, dirs, files in os.walk(path):
    for file in files:
        if file.startswith('sample_clusters'):
            signatures.append(re.split('sample_clusters_|_zscore', file)[1])
print(len(signatures))
print(signatures)

8
['6G_Interferon_Gamma', 'full', 'Gajewski_13G_Inflammatory', 'Inflammatory', 'Interferon_Gamma_Biology', 'Ribas_10G_Interferon_Gamma', 'selected', 'T-cell_Exhaustion']


In [10]:
# Only for Mondor 139
if series == 'mondor':
    for element in drop_list:
        signatures.remove(element)
    print(len(signatures))
    print(signatures)

6
['6G_Interferon_Gamma', 'Gajewski_13G_Inflammatory', 'Inflammatory', 'Interferon_Gamma_Biology', 'Ribas_10G_Interferon_Gamma', 'T-cell_Exhaustion']


In [11]:
for signature in signatures:
    df = pd.read_csv(os.path.join(path, 'sample_clusters_'+signature+'_zscore_hc_ward.D2_euclidean_3_reorder.csv'))
    df.rename(columns={'Cluster': signature}, inplace=True)
    print(df.shape)
    display(df.head(2))
    
    if signature == first_signature:
        df_res = df
    else:
        df_res = df_res.merge(df, 'outer')
print(df_res.shape)
display(df_res.head(2))

(139, 2)


,Sample,6G_Interferon_Gamma
0,EX128.ARN217,Cluster High
1,EX128.ARN279,Cluster High


(139, 2)


,Sample,Gajewski_13G_Inflammatory
0,EX128.ARN205,Cluster High
1,EX128.ARN329,Cluster High


(139, 2)


...

,Sample,Ribas_10G_Interferon_Gamma
0,EX128.ARN217,Cluster High
1,EX128.ARN322,Cluster High


(139, 2)


,Sample,T-cell_Exhaustion
0,EX128.ARN220,Cluster High
1,EX128.ARN286,Cluster High


(139, 7)


,Sample,6G_Interferon_Gamma,Gajewski_13G_Inflammatory,Inflammatory,Interferon_Gamma_Biology,Ribas_10G_Interferon_Gamma,T-cell_Exhaustion
0,EX128.ARN217,Cluster High,Cluster High,Cluster High,Cluster High,Cluster High,Cluster Median
1,EX128.ARN279,Cluster High,Cluster High,Cluster High,Cluster High,Cluster High,Cluster High


In [12]:
df_res.set_index('Sample', drop=True, inplace=True)
df_res.sort_values(by=list(df_res.columns.values), inplace=True)
print(df_res.shape)
display(df_res.head(5))
df_res.to_csv(os.path.join(path, 'summary_sample_clusters.csv'))

(139, 6)


,6G_Interferon_Gamma,Gajewski_13G_Inflammatory,Inflammatory,Interferon_Gamma_Biology,Ribas_10G_Interferon_Gamma,T-cell_Exhaustion
Sample,,,,,,
EX128.ARN279,Cluster High,Cluster High,Cluster High,Cluster High,Cluster High,Cluster High
EX128.ARN267,Cluster High,Cluster High,Cluster High,Cluster High,Cluster High,Cluster High
EX128.ARN250,Cluster High,Cluster High,Cluster High,Cluster High,Cluster High,Cluster High
EX128.ARN190,Cluster High,Cluster High,Cluster High,Cluster High,Cluster High,Cluster High
EX128.ARN220,Cluster High,Cluster High,Cluster High,Cluster High,Cluster High,Cluster High


In [13]:
samples_highs = df_res[(df_res=='Cluster High').sum(axis=1)==6].index.values
print(len(samples_highs))
print(samples_highs)
pd.DataFrame(samples_highs).to_csv(os.path.join(path, 'samples_always_cluster_high.csv'), index=False, header=False)

8
['EX128.ARN279' 'EX128.ARN267' 'EX128.ARN250' 'EX128.ARN190'
 'EX128.ARN220' 'EX128.ARN213' 'EX128.ARN286' 'EX128.ARN242']
